In [1]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
# define the templates
car_template =""" You are an expert in automobiles. You have extensive knowledge
        about car mechanics, model, repair works and automative technology. You provide
        clear and helpfu; answers about cars.
        Here is the question:
        Question: {question}
       """
technology_template =""" You are a tech expert with in-depth knowledge on the latest gadgets, software and technology trends
You provide insightful and detailed answers about technology.
 Here is the question:
        Question: {question}
"""
restaurant_template = """You are a knowledgeable food critic and restaurant reviewer. You have a deep understanding of \
different cuisines, dining experiences, and what makes a great restaurant. You answer questions about restaurants insightfully.

Here is the question:
{question}"""

In [3]:
# Create some sample questions 
car_questions =[
    "What is the difference between a sedan and a SUV?"
    "How does a hybrid car save fuel?"
    "What should I look for when buying a used car?"
]

restaurant_questions =[
    "What makes a five-star restaurant exceptional?"
    "What's meant by happy hours?"
    "what are the key elements of French cuisine?"
]
technology_questions =[
    "What is the difference between a smartphone and a laptop computer?"
    "What is the difference between a server and a client?"
    "How do I secure my mobile from cyber threats?"
]


In [4]:
embedding = OpenAIEmbeddings()
car_question_embeddings = embedding.embed_documents(car_questions)
technology_question_embeddings = embedding.embed_documents(technology_questions)
restaurant_question_embeddings = embedding.embed_documents(restaurant_questions)


In [5]:

def prompt_router(input):
    query_embedding = embedding.embed_query(input["question"])
   
    car_similarity = cosine_similarity([query_embedding], car_question_embeddings)[0]
    restaurant_similarity = cosine_similarity([query_embedding], restaurant_question_embeddings)[0]
    technology_similarity = cosine_similarity([query_embedding], technology_question_embeddings)[0]
   
    max_similarity = max(max(car_similarity), max(restaurant_similarity), max(technology_similarity))
    if max_similarity == max(car_similarity):
        print("Using CAR")
        return PromptTemplate.from_template(car_template)
    elif max_similarity == max(restaurant_similarity):
        print("Using RESTAURANT")
        return PromptTemplate.from_template(restaurant_template)
    elif max_similarity == max(technology_similarity):
        print("Using TECHNOLOGY")
        return PromptTemplate.from_template(technology_template)

input_query ={"question":"what's the best way to improve my laptop's  battery life?"}
prompt = prompt_router(input_query)

Using TECHNOLOGY


In [6]:
chain =(
    {"question":RunnablePassthrough()}
    | RunnableLambda(prompt_router)   # Get the dynamic prompt here
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)   
    | StrOutputParser()
)

In [7]:
chain.invoke("How to identify the best calori food item?")

Using RESTAURANT


'Identifying the best calorie-dense food item involves considering several factors, including nutritional value, flavor, and how well it fits into your dietary preferences or needs. Here are some key points to consider:\n\n1. **Nutritional Density**: Look for foods that provide a high number of calories along with essential nutrients. For example, nuts, seeds, avocados, and certain oils (like olive or coconut oil) are calorie-dense but also rich in healthy fats, vitamins, and minerals.\n\n2. **Flavor and Texture**: The best calorie-dense foods should not only be high in calories but also enjoyable to eat. Foods like dark chocolate, cheese, and creamy sauces can be both rich in calories and satisfying in flavor and texture.\n\n3. **Culinary Versatility**: Consider how the food can be used in various dishes. Ingredients like nut butters, full-fat dairy, and certain grains (like quinoa or farro) can be incorporated into a wide range of meals, enhancing both calorie content and taste.\n\n4

In [8]:
chain.invoke("Langchain or LamaIndex?")

Using TECHNOLOGY


"When comparing Langchain and LamaIndex, it's essential to understand the context in which each tool is used, as they serve different purposes within the realm of AI and natural language processing.\n\n### Langchain\n\n**Overview:**\nLangchain is a framework designed for building applications that utilize large language models (LLMs). It provides a structured way to integrate LLMs into various applications, enabling developers to create more complex and interactive experiences. Langchain focuses on chaining together different components, such as prompts, memory, and tools, to enhance the capabilities of LLMs.\n\n**Key Features:**\n1. **Modular Design:** Langchain allows developers to create modular applications by chaining together different components, making it easier to manage and scale applications.\n2. **Integration with External Tools:** It supports integration with various APIs and tools, enabling LLMs to interact with external data sources and services.\n3. **Memory Management:

In [9]:
chain.invoke("How to get good Mileage in driving?")

Using CAR


"Getting good mileage while driving involves a combination of driving habits, vehicle maintenance, and understanding your car's capabilities. Here are some tips to help you maximize fuel efficiency:\n\n### Driving Habits\n1. **Smooth Acceleration and Braking**: Avoid rapid acceleration and hard braking. Gradual changes in speed can improve fuel efficiency.\n2. **Maintain a Steady Speed**: Use cruise control on highways to maintain a constant speed, which can help save fuel.\n3. **Limit Idling**: Turn off your engine if you expect to be stopped for more than a minute or two. Idling consumes fuel without moving the vehicle.\n4. **Drive at Optimal Speeds**: Most cars achieve better fuel economy between 45-65 mph. Driving faster than this can significantly reduce mileage.\n5. **Plan Your Trips**: Combine errands into one trip to avoid multiple short trips, which can consume more fuel.\n\n### Vehicle Maintenance\n1. **Regular Maintenance**: Keep your car well-maintained with regular oil cha

## Using Classification

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
classification_template =PromptTemplate.from_template(
""" 
You are good at classifying a question.
Given the below question, classify it as either "Car" or "resturant" or "technology".

<If the question is about car mechanics, models, or automative technology,classify it as Car>
<If the question is about cuisines, dining experience, or resturant services, classify it as resturant>
<If the question is about technology, gadgets, or software, classify it as technology>

Question: {question}

Classification:
"""
)

In [11]:
classification_chain = classification_template | ChatOpenAI() | StrOutputParser()

In [12]:
def prompt_router(input_query):
    # find the classification for the query
    classification =classification_chain.invoke({"question":input_query["question"]})
    if classification == "Car":
        print("Using CAR")
        return PromptTemplate.from_template(car_template)
    elif classification == "Resturant":
        print("Using Resturant")
        return PromptTemplate.from_template(restaurant_template)
    elif classification == "Technology":
        print("Using Technology")
        return PromptTemplate.from_template(technology_template)
    else:
        print("Using Default", classification)
        return None

input_query = {"question": "What is the capital of France?"}
prompt = prompt_router(input_query)

Using Default N/A


In [13]:
input_query ={"question":"Can increasing the RAM will increase the laptop performance?"}
prompt = prompt_router(input_query)


Using Technology


In [14]:
chain =(
    {"question":RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    |ChatOpenAI()
    |StrOutputParser()
)

In [15]:
chain.invoke(".Net or python?")

Using Technology


'Answer: The decision between .NET and Python ultimately depends on your specific needs and preferences. \n\n.NET is a software framework developed by Microsoft that is widely used for building Windows applications. It is known for its robustness, scalability, and integration capabilities. .NET supports multiple programming languages such as C#, F#, and Visual Basic, making it versatile for developers. It also has a large community of developers and extensive documentation available.\n\nPython, on the other hand, is a versatile programming language that is popular for its simplicity, readability, and flexibility. It is widely used for web development, data analysis, artificial intelligence, and scientific computing. Python has a large number of libraries and frameworks that make it easy to build complex applications quickly.\n\nIf you are looking to develop Windows applications or work within the Microsoft ecosystem, .NET may be the better choice for you. However, if you are looking fo

## Using Tool Calling

In [16]:
from langchain_core.tools import tool

@tool
def classify_as_car(question: str) -> str:
    """Classifies the question as a 'Car' if it is about car mechanics, models or automotive technology.
    Args:
       question: The input question to be classified.
    Returns:
       A string 'Car' if the question is about car mechanics, models or automotive technology, otherwise None.
    """
    print("Funtion 'classify_as_car' was used")
    return 'Car'

@tool
def classify_as_resturant(question: str) -> str:
    """Classifies the question as a 'Restaurant' if it is about food, dining experiences, cuisines, drinks or resturant services.
    Args:
       question: The input question to be classified.
    Returns:
       A string 'Restaurant' if the question is about restaurants, food or drinks, otherwise None.
    """
    print("Funtion 'classify_as_restaurant' was used")
    return 'Restaurant'

@tool
def classify_as_technology(question: str) -> str:
    """Classifies the question as a 'Technology' if it is about computers, software, hardware, networks or other tech-related topics.
    Args:
       question: The input question to be classified.
    Returns:
       A string 'Technology' if the question is about computers, software, hardware, networks or other tech-related topics, otherwise None.
    """
    print("Funtion 'classify_as_technology' was used")
    return 'Technology'

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage

llm = ChatOpenAI()
query = "What is the latest trend in car market and what gadgets are relevant?"
#query = "How do I identify a good vintage wine at a resturant?"

tools = [classify_as_car, classify_as_resturant, classify_as_technology]
llm_with_tools = llm.bind_tools(tools)

messages =[HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
  selected_tool = {
    "classify_as_car": classify_as_car,
    "classify_as_resturant": classify_as_resturant,
    "classify_as_technology": classify_as_technology
  }[tool_call["name"].lower()]
  tool_output = selected_tool.invoke(tool_call["args"])
  messages.append(ToolMessage(content=tool_output, tool_call_id=tool_call["id"]))

messages

Funtion 'classify_as_car' was used
Funtion 'classify_as_technology' was used


[HumanMessage(content='What is the latest trend in car market and what gadgets are relevant?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sLOcO5b83KUkgiVM2QEENmU4', 'function': {'arguments': '{"question": "What is the latest trend in car market?"}', 'name': 'classify_as_car'}, 'type': 'function'}, {'id': 'call_oPvuW8wlcckGCVK1ifeTqa7R', 'function': {'arguments': '{"question": "What gadgets are relevant in the car market?"}', 'name': 'classify_as_technology'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 281, 'total_tokens': 346, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, i